In [1]:
import os
import wave
from warnings import warn
from audioop import add, mul
from IPython.display import Audio, display, HTML

### Dataset Creation (Echo)

In [2]:
def xdisplay(file): #display audio object
    return display(HTML('<a href='+file+'>'+file+'</a>'),
           Audio(url=file, embed=False))

def input_wave(filename,frames=10000000): #10000000 is an arbitrary large number of frames
    with wave.open(filename,'rb') as wave_file:
        params=wave_file.getparams()
        audio=wave_file.readframes(frames)  
    return params, audio

def output_wave(audio, params, stem, suffix): #dynamically format the filename by passing in data
    filename=stem.replace('.wav','_{}.wav'.format(suffix))
    with wave.open(filename,'wb') as wave_file:
        wave_file.setparams(params)
        wave_file.writeframes(audio)

def delay(audio_bytes,params,offset_ms,factor=1,num=1):
    if factor>1:
        warn("These settings may produce a very loud audio file. Please use caution when listening")
    offset=params.sampwidth*offset_ms*int(params.framerate/1000)
    audio_bytes=audio_bytes+b'\0'*offset*(num)
    delayed_bytes=audio_bytes
    for i in range(num):
        beginning = b'\0'*offset*(i+1)
        end = audio_bytes[:-offset*(i+1)]
        multiplied_end= mul(end,params.sampwidth,factor**(i+1))
        delayed_bytes= add(delayed_bytes, beginning+multiplied_end, params.sampwidth)
    return delayed_bytes

In [3]:
def EchoConverter(path):
    
    i = 0
    try: os.mkdir("Echo")
    except: pass
    
    Delx = [1, 50, 100, 300, 500, 1000]
    Volx = [0.1, 0.2, 0.4, 0.6, 0.8, 1]
    Mulx = [0, 1, 2, 3, 4, 5]
    
    for file in os.listdir(path):
        
        AudioParams, AudioBytes = input_wave(os.getcwd()+"\\"+path+"\\"+file)

        for D in Delx:
            
            for V in Volx:
                
                for M in Mulx:
                    
                    output_wave(delay(AudioBytes,AudioParams,D, V, M), AudioParams, 'Echo/' + file[:-4] + '.wav', 'D'+ str(D) + ' V' + str(V) + ' M' + str(M))
          
        i += 1
        
        print("Processing Audio: " + str(i), end="\r")

In [4]:
print('Enter the audio folder path') 
path = input()
EchoConverter(path)

Enter the audio folder path
Converted
